In [1]:
import pandas as pd 
import re as re # Regex

In [2]:
df = pd.read_csv('sewakost_40pages.tsv', sep="\t")

df.head()

,url,nama,harga,jenis,address,aircon,free_wifi,kamar_mandi_dalam,fasilitas_kamar,fasilitas_kost
0,https://www.sewakost.com/jakarta/selatan/kost-...,Kost Ciganjur Jalan Pasir No. 32,Rp 1.000.000,Kost Campur,Jalan Pasir no 32 Kp. Setu Rt 001/006 Keluraha...,Tidak,Tidak,Ya,['Kamar Mandi Dalam'],"['Keamanan', 'Parkir Motor']"
1,https://www.sewakost.com/jakarta/selatan/kost-...,KOST EXECUTIVE JAGAKARSA PASO CILANDAK JAKARTA...,Rp 2.200.000,Kost Putra,"Jalan Paso no 6A, Jagakarsa, Cilandak, Jakarta...",Ya,Ya,Ya,"['Spring Bed', 'Perabot', 'Free WiFi', 'AC', '...","['Ruang Tamu', 'Ruang Makan', 'Dapur', 'Air Mi..."
2,https://www.sewakost.com/jakarta/selatan/kamar...,Kamar kost Putri NON AC di Mampang,Rp 850.000,Kost Putri,Jl.Mampang Prapatan VI Rt04 Rw02 No.70B,Tidak,Tidak,Tidak,[],[]
3,https://www.sewakost.com/jakarta/selatan/hompy...,Hompy Timur Residence,Rp 2.700.000,Kost Putri,Tebet Timur Dalam 7B no. 11,Ya,Ya,Ya,"['Spring Bed', 'Perabot', 'Free WiFi', 'AC', '...","['Dapur', 'Cuci Setrika', 'Pembantu', 'Keamana..."
4,https://www.sewakost.com/jakarta/selatan/rumah...,Rumah kontrakan (40m2) Pasutri baru/karyawati/...,Rp 2.500.000,Kost Pasutri,Jalan keselamatan 18 RT10/RW03,Ya,Tidak,Ya,"['AC', 'Kamar Mandi Dalam']","['Ruang Tamu', 'Ruang Makan', 'Dapur', 'Keaman..."


In [3]:
# Terdapat kost dengan nilai NaN pada jenis dan fasilitas kost
df.isnull().sum()

url                   0
nama                  0
harga                 0
jenis                14
address              14
aircon               14
free_wifi            14
kamar_mandi_dalam    14
fasilitas_kamar      14
fasilitas_kost       14
dtype: int64

In [4]:
df[df.isnull().any(axis=1)].head()

,url,nama,harga,jenis,address,aircon,free_wifi,kamar_mandi_dalam,fasilitas_kamar,fasilitas_kost
483,https://www.sewakost.com/jakarta/selatan/indek...,INDEKOS ELITE DALAM CLUSTER FULL FURNISH,3.000.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
739,https://www.sewakost.com/jakarta/selatan/tebet...,TEBET EXECUTIVE RESIDENCE 141,3.550.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
740,https://www.sewakost.com/jakarta/selatan/sewa-...,"Sewa Kost, Cilandak Barat, Jakarta Selatan",2.000.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
741,https://www.sewakost.com/jakarta/selatan/kos-d...,Kos di Lt.Agung dkt KRL Lt.Agung & Tj.Brt,700.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
742,https://www.sewakost.com/jakarta/selatan/kost-...,Kost Wanita Tebet Dekat Pasar Tebet Barat dan ...,1.600.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Drop semua baris dengan nilai NaN karena tidak dapat digunakan untuk tahap analisis
df.dropna(inplace=True)

df.isnull().sum()

url                  0
nama                 0
harga                0
jenis                0
address              0
aircon               0
free_wifi            0
kamar_mandi_dalam    0
fasilitas_kamar      0
fasilitas_kost       0
dtype: int64

In [6]:
# Mengambil setiap nilai numerik kemudian mengubah ke dalam bentuk integer
def find_number(text):
    try:
        num = "".join(re.findall(r'[\d]+',text))
        return int(num)
    except:
        return 0

In [7]:
# Mengubah seluruh kolom harga menjadi integer
df['harga'] = df['harga'].apply(lambda x: find_number(x))

df.head()

,url,nama,harga,jenis,address,aircon,free_wifi,kamar_mandi_dalam,fasilitas_kamar,fasilitas_kost
0,https://www.sewakost.com/jakarta/selatan/kost-...,Kost Ciganjur Jalan Pasir No. 32,1000000,Kost Campur,Jalan Pasir no 32 Kp. Setu Rt 001/006 Keluraha...,Tidak,Tidak,Ya,['Kamar Mandi Dalam'],"['Keamanan', 'Parkir Motor']"
1,https://www.sewakost.com/jakarta/selatan/kost-...,KOST EXECUTIVE JAGAKARSA PASO CILANDAK JAKARTA...,2200000,Kost Putra,"Jalan Paso no 6A, Jagakarsa, Cilandak, Jakarta...",Ya,Ya,Ya,"['Spring Bed', 'Perabot', 'Free WiFi', 'AC', '...","['Ruang Tamu', 'Ruang Makan', 'Dapur', 'Air Mi..."
2,https://www.sewakost.com/jakarta/selatan/kamar...,Kamar kost Putri NON AC di Mampang,850000,Kost Putri,Jl.Mampang Prapatan VI Rt04 Rw02 No.70B,Tidak,Tidak,Tidak,[],[]
3,https://www.sewakost.com/jakarta/selatan/hompy...,Hompy Timur Residence,2700000,Kost Putri,Tebet Timur Dalam 7B no. 11,Ya,Ya,Ya,"['Spring Bed', 'Perabot', 'Free WiFi', 'AC', '...","['Dapur', 'Cuci Setrika', 'Pembantu', 'Keamana..."
4,https://www.sewakost.com/jakarta/selatan/rumah...,Rumah kontrakan (40m2) Pasutri baru/karyawati/...,2500000,Kost Pasutri,Jalan keselamatan 18 RT10/RW03,Ya,Tidak,Ya,"['AC', 'Kamar Mandi Dalam']","['Ruang Tamu', 'Ruang Makan', 'Dapur', 'Keaman..."


In [8]:
df.describe()

,harga
count,9.440000e+02
mean,3.816841e+09
std,8.280764e+10
min,0.000000e+00
25%,1.000000e+06
50%,1.650000e+06
75%,2.350000e+06
max,1.800000e+12


In [9]:
# Drop harga kos di bawah 300rb dan di atas 5 juta
df = df[(df['harga']>=300000) & (df['harga']<=5000000)]

df.shape

(872, 10)

In [10]:
df['jenis'].unique()

array(['Kost Campur', 'Kost Putra', 'Kost Putri', 'Kost Pasutri',
       'Kost Keluarga'], dtype=object)

In [11]:
df.dtypes

url                  object
nama                 object
harga                 int64
jenis                object
address              object
aircon               object
free_wifi            object
kamar_mandi_dalam    object
fasilitas_kamar      object
fasilitas_kost       object
dtype: object

In [12]:
# Mengubah jenis kost pasutri -> kost keluarga
df['jenis'] = df['jenis'].replace('Kost Pasutri', 'Kost Keluarga')

# Mengubah ke dalam bentuk kategorikal
categorical = ['jenis', 'aircon', 'free_wifi', 'kamar_mandi_dalam']
df[categorical] = df[categorical].astype('category')

df.dtypes

url                    object
nama                   object
harga                   int64
jenis                category
address                object
aircon               category
free_wifi            category
kamar_mandi_dalam    category
fasilitas_kamar        object
fasilitas_kost         object
dtype: object

1. Get unique in `fasilitas_kamar`
2. Looping dan ekstrak setiap fasilitas
3. Fasilitas dibuat menjadi kolom baru
4. Iterasi setiap baris pada dataframe dan cek kolom fasilitas kamar (iterasi lagi per kolom fasilitas yang baru) pakai `is in` maybe. Jika ada maka `Yes` atau `1` di kolom per fasilitas.

In [47]:
def get_fasilitas(data_fasilitas):
    fasilitas = []
    for n in data_fasilitas:
        data = n[1:-1]
        if data == "":
            continue
        list_fas = data.replace("'", "").split(", ")
        for fas in list_fas:
            if fas not in fasilitas:
                fasilitas.append(fas)
    
    return fasilitas

In [51]:
fasilitas_kamar = get_fasilitas(df['fasilitas_kamar'].unique())
print(fasilitas_kamar)

fasilitas_kost = get_fasilitas(df['fasilitas_kost'].unique())
print(fasilitas_kost)

['Kamar Mandi Dalam', 'Spring Bed', 'Perabot', 'Free WiFi', 'AC', 'TV', 'Pemanas Air (Water Heater)', 'Kulkas']
['Keamanan', 'Parkir Motor', 'Ruang Tamu', 'Ruang Makan', 'Dapur', 'Air Minum', 'Pembantu', 'Parkir Mobil', 'Cuci Setrika', 'Mesin Cuci (Cuci Sendiri)', 'Televisi']
